In [1]:
import pandas as pd
import numpy as np

In [2]:
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [3]:
import h5py
h5f = h5py.File('dataset.h5', 'r')
X  = h5f['X']
Y = h5f['Y']

In [4]:
Y

<HDF5 dataset "Y": shape (8000, 5), type "<f4">

In [5]:
# Shuffle the data
X, Y = shuffle(X, Y)

In [6]:
y=pd.DataFrame(Y)
y=y.iloc[:,1:]
y=np.array(y)

In [7]:
pd.DataFrame(y).describe()

,0,1,2,3
count,8000.000000,8000.000000,8000.000000,8000.000000
mean,0.434875,0.232000,0.107375,0.225750
std,0.495772,0.422135,0.309609,0.418101
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [8]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center(mean=0.457434658462)
img_prep.add_featurewise_stdnorm(std=0.209459554323)

In [9]:
# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

In [10]:
# Input is a 128x128 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 128, 128, 3],
                     data_preprocessing=img_prep,
data_augmentation=img_aug)

In [11]:
X.shape

(8000, 128, 128, 3)

In [12]:
Y.shape

(8000, 5)

In [13]:
# Step 1: Convolution
network = conv_2d(network, 128, 3, activation='relu')

In [14]:
# Step 2: Max pooling
network = max_pool_2d(network, 2)

In [15]:
# Step 3: Convolution again
network = conv_2d(network, 256, 3, activation='relu')

In [16]:
# Step 4: Convolution yet again
network = conv_2d(network, 256, 3, activation='relu')

In [17]:
# Step 5: Max pooling again
network = max_pool_2d(network, 2)

In [18]:
# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

In [19]:
# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

In [20]:
# Step 8: Fully-connected neural network with two outputs (0=isn't a bird, 1=is a bird) to make the final prediction
network = fully_connected(network, 4, activation='softmax')

In [21]:
# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy', n_classes=4, learning_rate=0.1)

In [22]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='bird-classifier.tfl.ckpt')


In [ ]:
model.fit(X, y, n_epoch=2, shuffle=True, #validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96,
          snapshot_epoch=True, validation_set=0.3,
run_id='bird-classifier')

In [ ]:
# Save model when training is complete to a file
model.save("bird-classifier.tfl")
print("Network trained and saved as bird-classifier.tfl!")

In [35]:
y

array([[ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.]], dtype=float32)

In [7]:
import scipy

In [ ]:
import pandas